# 3. Deep Learning

### Definición RNN
Usar capa SimpleRNN y la columna demanda como target.

In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px

from encoding import encoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_error, r2_score
from holidays import Spain

from keras.layers import Input, SimpleRNN, Dense
from keras.models import Sequential
from keras.optimizers import Adam


In [2]:
ruta_archivo = "..\data\processed\DF_DEMANDA_10_25_LIMPIO.csv"
df = pd.read_csv(ruta_archivo)

## Preparación de los datos

#### Filtrar datos
Escogemos las columnas de interés y pasamos MWh a GWh

In [3]:
df_filtrado = df[(df['zona'] == 'nacional') & (df['titulo'] == 'Demanda')].sort_values(by='fecha').reset_index(drop=True)
df_filtrado = df_filtrado[['valor_(MWh)', 'año', 'mes', 'dia', 'dia_semana']].reset_index(drop=True)
df_filtrado['valor_(MWh)'] = df_filtrado['valor_(MWh)'].apply(lambda x:round(x*0.001, 3))
df_filtrado.rename(columns={'valor_(MWh)': 'valor_(GWh)'}, inplace=True)

In [4]:
df_filtrado

,valor_(GWh),año,mes,dia,dia_semana
0,605.986,2011,1,1,sábado
1,641.856,2011,1,2,domingo
2,801.297,2011,1,3,lunes
3,833.253,2011,1,4,martes
4,803.476,2011,1,5,miércoles
...,...,...,...,...,...
5170,708.223,2025,2,26,miércoles
5171,733.080,2025,2,27,jueves
5172,723.098,2025,2,28,viernes
5173,657.051,2025,3,1,sábado


### Encoding
Vamos a usar un encoding circular, para que el modelo entienda mejor la estacionalidad de los datos.

Primero, definimos los días que tiene cada mes. Debemos hacerlo así, con un diccionario, ya que si intentamos hacer un groupby para sacar cuántos días tiene cada mes en el histórico (viendo los bisiestos), del mes actual solo cogerá el número de días que hayan pasado (por ejemplo, si tenemos datos hasta el 6 de marzo, escalará los datos de ese mes dividiendo entre 6...)

In [5]:
df_filtrado = encoder(df_filtrado)

In [6]:
df_filtrado 

,valor_(GWh),año,dia_semana_sin,dia_semana_cos,mes_sin,mes_cos,dia_mes_sin,dia_mes_cos
0,605.986,2011,-0.974928,-0.222521,0.500000,8.660254e-01,2.012985e-01,0.979530
1,641.856,2011,-0.781831,0.623490,0.500000,8.660254e-01,3.943559e-01,0.918958
2,801.297,2011,0.000000,1.000000,0.500000,8.660254e-01,5.712682e-01,0.820763
3,833.253,2011,0.781831,0.623490,0.500000,8.660254e-01,7.247928e-01,0.688967
4,803.476,2011,0.974928,-0.222521,0.500000,8.660254e-01,8.486443e-01,0.528964
...,...,...,...,...,...,...,...,...
5170,708.223,2025,0.974928,-0.222521,0.866025,5.000000e-01,-4.338837e-01,0.900969
5171,733.080,2025,0.433884,-0.900969,0.866025,5.000000e-01,-2.225209e-01,0.974928
5172,723.098,2025,-0.433884,-0.900969,0.866025,5.000000e-01,-2.449294e-16,1.000000
5173,657.051,2025,-0.974928,-0.222521,1.000000,6.123234e-17,2.012985e-01,0.979530


Si lo representamos, sale un círculo de radio 1.

In [7]:
fig = px.line(
    data_frame=df_filtrado[:32],
    x='dia_mes_sin',
    y='dia_mes_cos'
)

fig.show()

### Escalado de datos

In [8]:
scaler = MinMaxScaler()
cols_to_scale = ["valor_(GWh)", "año"]  

df_filtrado[cols_to_scale] = scaler.fit_transform(df_filtrado[cols_to_scale]) 

### Secuencias de entrada y salida
Las redes recurrentes aprenden observando una secuencia con sus características y prediciendo el siguiente valor del target (en este caso, la demanda). Lo que haremos será crear ventanas deslizantes de "loockback" días:
- En la X nos guardamos los datos de los 'loockback' días anteriores.
- En la y intentará predecir el día siguiente.
- Devuelve un array para cada una que podrá entrar a la red neuronal.

In [9]:
def create_sequences(df, target_column, lookback):
    X, y = [], []
    for i in range(len(df) - lookback):
        X.append(df.iloc[i:i+lookback].drop(columns=[target_column]).values) 
        y.append(df.iloc[i+lookback][target_column]) 
    return np.array(X), np.array(y)

# Definir la ventana de tiempo - 14 días
lookback = 14  

X, y = create_sequences(df_filtrado, target_column="valor_(GWh)", lookback=lookback)

print("Forma de X:", X.shape)  # (n_samples, lookback, n_features)
print("Forma de y:", y.shape)  # (n_samples,)

Forma de X: (5161, 14, 7)
Forma de y: (5161,)


### Train/Test
Debe mantener la temporalidad: el 80% de los datos más antiguos irán al train set y el 20% restante al test

In [10]:
def train_test(f=0.8):    
    train_size = int(len(X) * f)

    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = train_test()

## Modelo RNN
Generamos la estructura de la red neuronal. 
+ En primer lugar usaremos una RNN simple y veremos la función de pérdida.
+ Luego aplicaremos una

### Definición RNN simple

In [11]:
model = Sequential()

# Capa de Entrada - usamos 14 días y le damos el número de columnas independientes
model.add(Input(shape = (lookback, X.shape[2])))

model = Sequential([
    SimpleRNN(64, activation="relu"),  # Capa recurrente
    Dense(32, activation="relu"),  # Capas ocultas
    Dense(1)  # Capa de salida para predecir la demanda
])

model.compile(optimizer = "adam", loss = "mse")

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(x = X_train,
                    y = y_train,
                    validation_data = (X_val, y_val),
                    epochs = 20,
                    verbose=1)

Epoch 1/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.2341 - val_loss: 0.0204
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0108 - val_loss: 0.0124
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0095 - val_loss: 0.0173
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0087 - val_loss: 0.0122
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0078 - val_loss: 0.0135
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0080 - val_loss: 0.0115
Epoch 7/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0071 - val_loss: 0.0135
Epoch 8/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0075 - val_loss: 0.0111
Epoch 9/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0073 - val_loss: 0.0112
Epoch 10/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0068 - val_loss: 0.0113
Epoch 11/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0068 - val_loss: 0.0080
Epoch 12/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

In [24]:
fig = px.line(data_frame=history.history,
        y=['loss', 'val_loss'],
        title='Función de pérdida (loss) basada en el MSE',
        labels={'index': 'Época', 'value': 'Pérdida'},
        )

fig.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig.for_each_trace(lambda t: t.update(name="Pérdida Entrenamiento" if t.name == "loss" else "Pérdida Validación"))

fig.show()

### One-step predictions
Aquí nos cogeremos los datos de los últimos 14 días y con ellos haremos la predicción de los siguientes 14 de uno en uno.

In [14]:
N = len(y)
validation_target = y[-N//2:]
validation_predictions = []

i = -N//2

while len(validation_predictions) < len(validation_target):
    
    # Predice el siguiente valor de X[i]
    p = model.predict(X[i].reshape(1, X.shape[1], X.shape[2]))[0, 0]
    i += 1
    
    validation_predictions.append(p)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━

In [15]:
def desescalado(val_target, val_pred):

    # Crear un array con la misma cantidad de columnas que el scaler espera
    dummy_features = np.zeros((len(val_pred), 1))

    # Convertir las predicciones en un array con la forma adecuada
    val_pred = np.array(val_pred).reshape(-1, 1)
    val_target = np.array(val_target).reshape(-1, 1)

    # Unir las predicciones con los valores ficticios
    predictions_with_dummy = np.hstack([val_pred, dummy_features])
    validation_target_dummy = np.hstack([val_target, dummy_features])

    # Aplicar la transformación inversa
    predictions_real = scaler.inverse_transform(predictions_with_dummy)[:, 0]  # Solo tomar la columna de interés
    validation_real = scaler.inverse_transform(validation_target_dummy)[:, 0]

    return validation_real, predictions_real

In [16]:
validation_real, predictions_real = desescalado(validation_target, validation_predictions)

In [17]:
fig_one_step = px.line(
                       y=[validation_real, predictions_real],
                       title='Predicción de la demanda en 14 días',
                       labels={'index': 'Día', 'value': 'Demanda (GWh)'},
                       )

fig_one_step.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_one_step.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_one_step.show()

In [18]:
mse = round(mean_squared_error(validation_real, predictions_real),2)
print(f"MSE: {mse}")

mae = round(mean_absolute_error(validation_real, predictions_real), 2)
print(f"MAE: {mae}")

rmse = round(np.sqrt(mse), 2)
print(f"RMSE: {rmse}")

r2 = round(r2_score(validation_real, predictions_real), 2)
print(f"R²: {r2}")

MSE: 1357.05
MAE: 28.06
RMSE: 36.84
R²: 0.73


### Multi-step prediction
Aquí, también haremos la predicción de los siguientes 14 días, pero aprovechando en cada iteración los nuevos datos aportados por la predicción anterior.

In [19]:
N = len(y)
validation_target_multi = y[-N//2:]  # Última mitad de los datos reales
validation_predictions_multi = []

# última ventana
last_x = X[-14]

while len(validation_predictions_multi) < len(validation_target_multi):
    
    # En la primera iteración predice el siguiente valor de y usando X
    # En las siguientes iteraciones usa el valor predicho anterior para predecir el siguiente
    p = model.predict(last_x.reshape(1, X.shape[1], X.shape[2]))[0, 0]
    print(p)
    
    validation_predictions_multi.append(p)
    print(f"Valor: {last_x[-1][0]}\tPredicción: {p}")

    # Desplaza los elementos en last_x hacia atrás, dejando el primer elemento al final
    last_x = np.roll(last_x, -1, axis=0)
    
    # Cambia el último elemento a la predicción
    last_x[-1, 0] = p

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
0.5860235
Valor: 1.0	Predicción: 0.5860235095024109
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
0.57181996
Valor: 0.5860235095024109	Predicción: 0.5718199610710144
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
0.5860823
Valor: 0.5718199610710144	Predicción: 0.5860822796821594
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
0.6983218
Valor: 0.5860822796821594	Predicción: 0.6983218193054199
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
0.65965086
Valor: 0.6983218193054199	Predicción: 0.6596508622169495
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
0.55135864
Valor: 0.6596508622169495	Predicción: 0.5513586401939392
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
0.41677558
Valor: 0.5513586401939392	Predicción: 0.41677558422088623
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
0.6792942
Valor: 0.41677558422088623	Predicción: 0.679294228553772
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
0.7086602
Valor: 0.679294228553772	Predicción: 0.7086601853370667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
0.73798996
Valor: 0.70866018

In [20]:
validation_real_multi, predictions_real_multi = desescalado(validation_target, validation_predictions_multi)

In [21]:
fig_multi = px.line(
                       y=[validation_real_multi, predictions_real_multi],
                       title='Predicción de la demanda en 14 días',
                       labels={'index': 'Día', 'value': 'Demanda (GWh)'},
                       )

fig_multi.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_multi.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_multi.show()

In [22]:
mse = round(mean_squared_error(validation_real_multi, predictions_real_multi),2)
print(f"MSE: {mse}")

mae = round(mean_absolute_error(validation_real_multi, predictions_real_multi), 2)
print(f"MAE: {mae}")

rmse = round(np.sqrt(mse), 2)
print(f"RMSE: {rmse}")

r2 = round(r2_score(validation_real_multi, predictions_real_multi), 2)
print(f"R²: {r2}")

MSE: 14628.99
MAE: 98.73
RMSE: 120.95
R²: -1.87


### Función con ruido
Vamos a añadir algo de ruido a las predicciones, de forma que no sean tan rígidas y puedan ajustarse mejor a variabilidades no explicadas.

In [23]:
N = len(y)
validation_target_noise = y[-N//2:] 
validation_predictions_noise = []

last_x = X[-14]

sigma = 0.001

while len(validation_predictions_noise) < len(validation_target_noise):   
    # Predice el siguiente valor
    p = model.predict(last_x.reshape(1, 14, X.shape[2]))[0, 0]
    
    # Agregar ruido gaussiano
    noise = np.random.normal(loc=0, scale=sigma, size=1)[0]
    p_noisy = p + noise
    
    validation_predictions_noise.append(p_noisy)
    print(f"Valor real: {validation_target_noise[_]}\tPredicción con ruido: {p_noisy}")

    # Desplaza en el eje temporal
    last_x = np.roll(last_x, -1, axis=0)

    # Reemplaza el último valor predicho en la columna de salida
    last_x[-1, 0] = p_noisy  # Suponiendo que la salida está en la primera columna


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
validation_real_noise, predictions_real_noise = desescalado(validation_target_noise, validation_predictions_noise)

In [ ]:
fig_noise = px.line(
                       y=[validation_real_noise, predictions_real_noise],
                       title='Predicción de la demanda en 14 días',
                       labels={'index': 'Día', 'value': 'Demanda (GWh)'},
                       )

fig_noise.update_layout(
    title_x=0.5,
    legend_title_text="Variables"
)

fig_noise.for_each_trace(lambda t: t.update(name="Demanda real" if t.name == "wide_variable_0" else "Predicción"))

fig_noise.show()

In [ ]:
mse = round(mean_squared_error(validation_real_noise, predictions_real_noise),2)
print(f"MSE: {mse}")

mae = round(mean_absolute_error(validation_real_noise, predictions_real_noise), 2)
print(f"MAE: {mae}")

rmse = round(np.sqrt(mse), 2)
print(f"RMSE: {rmse}")

r2 = round(r2_score(validation_real_noise, predictions_real_noise), 2)
print(f"R²: {r2}")

MSE: 11304.45
MAE: 87.98
RMSE: 106.32
R²: -1.21
